# Metric Learning - LMNN

Underneath you can define the hyperparameters, l, mu and K.
* l is the margin parameter
* mu is a trade-off parameter between the push and pull in the loss function
* K is the number of target neighbors 

In [39]:
import sklearn
import numpy as np
from Model import get_data

l = 1
mu = 1
K = 2

* F is the dimensionality of our data. We choose that arbitrarily
* L is the linear transformation

L is set to be a diagonal matrix of ones to begin with. However, it would be interesting to experiement with other initial L matrices, since the problem is non-convex. We could also try to implement the constrained non-convex version of LMNN.

In [40]:
F = 128
L = np.eye(F)

## Data Import & Analysis

In [46]:
embeddings, labels = get_data()
s_score = sklearn.metrics.silhouette_score(embeddings.T, labels, metric='euclidean')

print("Shape of embeddings is: {}".format(np.shape(embeddings)))
print("Shape of labels is: {} \n".format(np.shape(labels)))
print("Silhouette Score of data is: {}".format(s_score))

Shape of embeddings is: (128, 100)
Shape of labels is: (100,) 

Silhouette Score of data is: 0.2929665148258209


### TODO: Plots

We should put in some plots of the data here.. We will need that for the presentation

## Functions

Chi square distance, as described in equation (3) in the non-linear metric learning paper

In [18]:
def chi_square_distance(xi, xj):
    """
    Chi square distance

    :param xi: Embedding (1, F)
    :param xj: Target Neighbor (1, F)
    :return: Distance
    """
    return 1/2 * np.sum(np.square(xi - xj) / (xi + xj))

Loss function as described in equaltion (5) in the paper

In [12]:
def loss_function(xi, xj, xk):
    """
    Loss function as described in the paper

    :param xi: One embedding        (1, F)
    :param xj: K target neighbors   (K, F)
    :param xk: Unknown imposters    (?, F)

    :return: Loss for one embedding
    """

    _, K = np.shape(xj)
    imposter, _ = np.shape(xj)
    sum = 0

    for j in range(K):
        sum += chi_square_distance(L @ xi, L @ xj[:, j])
        inner_sum = 0
        for k in range(imposter):
            inner_sum += max(0, l + chi_square_distance(xi, xj[:, j]) - chi_square_distance(xi, xk[k, :]))
        sum += mu * inner_sum
    return sum


Functions to find triples. Distance function need to be changed, to find the distance through the L plane, and not just in the euclidean space. 

In [59]:
def distance(xi, X):
    """
    :param xi: Embedding vector (1, F)
    :param X: Data matrix without embedding vector (N, F)
    :return: Distance vector (1, N)
    """
    return np.linalg.norm(((X @ L) - (xi @ L)), axis=1)


def find_triplets(xi, yi, X, y):
    """
    Given some vector xi and corresponding label yi, find target neighbors and imposters

    :param xi: Embedding vector (1, F)
    :param yi: Label for embedding vector
    :param X: Full data matrix  (N, F)
    :return: target_neighbors and imposters for embedding (K, F) (?, F)
    """
    candidate_target_neighbors = X[np.where(yi == y)]
    imposters = X[np.where(yi != y)]

    target_neighbors_dist = distance(xi, candidate_target_neighbors)
    imposters_dist = distance(xi, imposters)

    # Find K target neighbors
    target_neighbors = np.zeros((1, F))
    for i in range(K):
        min_index = np.argmin(target_neighbors_dist)
        target_neighbors = np.vstack((target_neighbors, candidate_target_neighbors[min_index]))
        candidate_target_neighbors = np.delete(candidate_target_neighbors, (min_index), axis=0)
    target_neighbors = target_neighbors[1:, :]

    # Find ? imposters
    max_target_dist = np.max(target_neighbors_dist)
    imposters = imposters[np.where(imposters_dist < max_target_dist + l)]

    return target_neighbors, imposters

## Run LMNN

In [73]:
# TODO:  Run this in loop over all vectors in X 
xj, xk = find_triplets(xi, yi, X, Y)
loss_function(xi.T, xj.T, xk.T)

6.361904761904762